In [1]:
import pandas as pd
import numpy as np
from ipywidgets import IntProgress
from matplotlib import pyplot as plt

In [2]:
dif = .1
start = 0.
Fs = [{'label': 'f1', 'F': 1.}]
for i in range(1, 10):
    cur = round(start+dif*i, 2)
    Fs.append({'label':'f'+str(i+1), 'F':cur})

base=.9
dif = .9
for i in range(0, 5):
    dif = dif*.1
    base = base + dif
    cur = round(base, i+2)
    Fs.append({'label':'f'+str(i+11), 'F':cur})

In [3]:
Fs

[{'label': 'f1', 'F': 1.0},
 {'label': 'f2', 'F': 0.1},
 {'label': 'f3', 'F': 0.2},
 {'label': 'f4', 'F': 0.3},
 {'label': 'f5', 'F': 0.4},
 {'label': 'f6', 'F': 0.5},
 {'label': 'f7', 'F': 0.6},
 {'label': 'f8', 'F': 0.7},
 {'label': 'f9', 'F': 0.8},
 {'label': 'f10', 'F': 0.9},
 {'label': 'f11', 'F': 0.99},
 {'label': 'f12', 'F': 0.999},
 {'label': 'f13', 'F': 0.9999},
 {'label': 'f14', 'F': 0.99999},
 {'label': 'f15', 'F': 0.999999}]

In [4]:
chunkMapDF = pd.read_csv('data/ChunkMap.csv')
# qrVals = pd.read_csv('QR_Values.csv')

In [5]:
var = 0

In [6]:
H_TRANSPOSE=None
H = None

def leastSquare(X, Y, F, SCALED_DATA=False):
    global H, H_TRANSPOSE
    H_TRANSPOSE = np.vstack(([1.]*len(X), X))
    H = np.transpose(H_TRANSPOSE).copy()
    currF = 1.
    i=0
    currF = F
    for x in X:
        if(SCALED_DATA):
            currF = F**(100-x)
        else:
            currF = currF*F
        H_TRANSPOSE[0][i], H_TRANSPOSE[1][i] = H_TRANSPOSE[0][i]*currF, H_TRANSPOSE[1][i]*currF
        i+=1
    #We are using np.linalg.lstsq because unlike .inv and .solve, .lstsq accepts singular matrices
    if(np.linalg.det(H_TRANSPOSE @ H) == 0):
        return np.linalg.lstsq(H_TRANSPOSE @ H, H_TRANSPOSE @ Y)[0]
    else:
        return np.linalg.solve(H_TRANSPOSE @ H, H_TRANSPOSE @ Y)


def interpXY(X, Y):
    numPoints = pd.Series(X).max()+1
    startX = X[0]
    newX = list(range(startX, numPoints+startX))
    newY = [np.nan]*numPoints    
    for i in range(0, len(X)):
        xV = X[i]-startX
#         print(i, xV)
        newY[xV] = Y[i]
    newY = list(pd.Series(newY).interpolate())
    return newX, newY

def raiseAB(A, B):
    return np.exp(A), B

def genABData(A, B):
#     X = list(range(0, Xf))
    X, Y = [], []
    for i in range(0, 100):
        x = i
        X.append(x)
        Y.append(A*np.exp(B*x))
    return X, Y

def genData(A, B):
#     X = list(range(0, Xf))
    X, Y = [], []
    for i in range(0, 100):
        x = i
        X.append(x)
        Y.append(A+B*x)
    return X, Y

def processRegr(X, Y, F, SCALED_DATA=False):
    A, B = leastSquare(X, np.log(Y), F, SCALED_DATA=SCALED_DATA)
    if(A=="HTooShort"):
        return A, B
    else:
        return raiseAB(A, B)


In [16]:
scaled = False

In [17]:
chunkNames = chunkMapDF['Chunk'].unique()[:]

chunkBar = IntProgress(min=0, max=len(chunkNames))
contestBar = IntProgress(min=0, max=10000)

print("Chunks:")
display(chunkBar)

print("Contests:")
display(contestBar)

results = []

for chunkName in chunkNames[:]:
    print(chunkName)
    if(scaled):
        chunkDF = pd.read_csv('data/Chunks_Scaled/'+chunkName+'.csv')
    else:
        chunkDF = pd.read_csv('data/Chunks/'+chunkName+'.csv')

    contests = chunkDF['ContestId'].unique()[:]
    contestBar.value=0
    contestBar.max = len(contests)
    for cid in contests[:]:
        if(scaled):
            cSeriesDF = chunkDF[chunkDF['ContestId']==cid]
            cSeriesDF = cSeriesDF[cSeriesDF['Before4HoursOut']]
            X = cSeriesDF['TimeScaled']
            Y = cSeriesDF['EntriesScaled']
            X, Y = list(X), list(Y)        
            count = 0
        else:
            cSeriesDF = chunkDF[chunkDF['ContestId']==cid].sort_values(by=['MinutesRemaining'], ascending=False)
            numberToRemove = len(cSeriesDF[cSeriesDF['MinutesRemaining']<240])
            X = cSeriesDF['MinutesRemaining'].max() - cSeriesDF['MinutesRemaining'][:-numberToRemove]
            Y = cSeriesDF['Entries'].cumsum()[:-numberToRemove]
            X, Y = list(X), list(Y)   
        
        contest = {'ContestId':cid}
        
        for fFactor in Fs[:]:
            fName = fFactor['label']
            fFactor = fFactor['F']
            A, B = 'TooShort', 'TooShort'
            if(len(X)>=1):
                 A, B  = processRegr(X, Y, fFactor, SCALED_DATA=scaled)
            contest[fName+'A'], contest[fName+'B']=A, B    
        results.append(contest)
        contestBar.value+=1
    chunkBar.value+=1


Chunks:


IntProgress(value=0, max=65)

Contests:


IntProgress(value=0, max=10000)

chunk1


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.


chunk2
chunk3
chunk4
chunk5
chunk6
chunk7
chunk8
chunk9
chunk10
chunk11
chunk12
chunk13
chunk14
chunk15
chunk16
chunk17
chunk18
chunk19
chunk20
chunk21
chunk22
chunk23
chunk24
chunk25
chunk26
chunk27
chunk28
chunk29
chunk30
chunk31
chunk32
chunk33
chunk34
chunk35
chunk36
chunk37
chunk38
chunk39
chunk40
chunk41
chunk42
chunk43
chunk44
chunk45
chunk46
chunk47
chunk48
chunk49
chunk50
chunk51
chunk52
chunk53
chunk54
chunk55
chunk56
chunk57
chunk58
chunk59
chunk60
chunk61
chunk62
chunk63
chunk64
chunk65


In [18]:
resultsDF = pd.DataFrame(results).set_index('ContestId')

In [19]:
len(resultsDF)

649329

In [20]:
len(resultsDF[resultsDF['f1A']=='TooShort'])

294389

scaled had 198k TooShort

In [22]:
if(scaled):
    resultsDF.to_csv('data/scaled/LR_Values.csv')
else:
    resultsDF.to_csv('data/unscaled/LR_Values.csv')